In [4]:
import requests 
from bs4 import BeautifulSoup 
from timeit import timeit
import numpy as np
import random
import pandas as pd
# import numpy as np
# URL = "https://www.espncricinfo.com/ci/content/squad/index.html?object=1210595"
# r = requests.get(URL) 

# soup = BeautifulSoup(r.content, 'html5lib') # If this line causes an error, run 'pip install html5lib' or install html5lib 

In [7]:
def scraping_fun(URL):
    
    r = requests.get(URL) 

    soup = BeautifulSoup(r.content, 'html5lib')
    
    
    
    ipl_team=['Chennai Super Kings (2020/21)','Delhi Capitals Squad','Kings XI Punjab Squad','Kolkata Knight Riders Squad',
          'Mumbai Indians Squad','Rajasthan Royals Squad','Royal Challengers Bangalore Squad','Sunrisers Hyderabad Squad']
    
    a_tag=soup.find_all('a')
    
    dictx={}
    team=[]
    url=[]
    for a in a_tag:
        if a.text in ipl_team:
            team.append(a.text)
            url.append(a['href'])
    dictx={'team':team,'Squad':url}
    
    Ipl_squad_url=pd.DataFrame.from_dict(dictx)
    
    Ipl_squad_url['team']=np.where(Ipl_squad_url['team']=='Chennai Super Kings (2020/21)','Chennai Super Kings',Ipl_squad_url['team'])
    Ipl_squad_url['team']=np.where(Ipl_squad_url['team']=='Delhi Capitals Squad','Delhi Capitals',Ipl_squad_url['team'])
    Ipl_squad_url['team']=np.where(Ipl_squad_url['team']=='Kings XI Punjab Squad','Kings XI Punjab',Ipl_squad_url['team'])
    Ipl_squad_url['team']=np.where(Ipl_squad_url['team']=='Kolkata Knight Riders Squad','Kolkata Knight Riders',Ipl_squad_url['team'])
    Ipl_squad_url['team']=np.where(Ipl_squad_url['team']=='Mumbai Indians Squad','Mumbai Indians',Ipl_squad_url['team'])
    Ipl_squad_url['team']=np.where(Ipl_squad_url['team']=='Rajasthan Royals Squad','Rajasthan Royals',Ipl_squad_url['team'])
    Ipl_squad_url['team']=np.where(Ipl_squad_url['team']=='Royal Challengers Bangalore Squad','Royal Challengers Bangalore',Ipl_squad_url['team'])
    Ipl_squad_url['team']=np.where(Ipl_squad_url['team']=='Sunrisers Hyderabad Squad','Sunrisers Hyderabad ',Ipl_squad_url['team'])
    
    Ipl_squad_url['team']=Ipl_squad_url['team'].str.lower().str.strip()
    
    return Ipl_squad_url

In [8]:
URL = "https://www.espncricinfo.com/ci/content/squad/index.html?object=1210595"
Ipl_squad_url=scraping_fun(URL)

In [9]:
def squad_looping(team,df=Ipl_squad_url):
    url=df[df['team'] == team.lower()]['Squad']
    return url.values[0]

In [16]:
def team_URL(team_name):
    URL = "https://www.espncricinfo.com/"+squad_looping(team_name)
    return URL

In [17]:
URL=team_URL('chennai super kings')
# print(URL)

https://www.espncricinfo.com//ci/content/squad/1210601.html


In [18]:
def team_details_scraping(URL):
    r = requests.get(URL) 
    soup = BeautifulSoup(r.content, 'html5lib') 
    
    player_name=soup.find_all('div',{"class": "large-13 medium-13 small-13 columns"})
    
    player_name_no_img=soup.find_all('div',{"class": "large-13 medium-13 small-20 columns"})
    
    name=[]
    player_status=[]
    player_stat=[]
    for player in player_name:
        name.append(player.a.text.strip()) #player.h3.text
        player_stat.append("https://www.espncricinfo.com/"+ player.h3.a['href'])
        try:
            player_status.append(player.h3.span.text)
        except:
            player_status.append('Indian player')
    for player in player_name_no_img:
        name.append(player.a.text.strip()) #player.h3.text
        player_stat.append("https://www.espncricinfo.com"+ player.h3.a['href'])
        try:
            player_status.append(player.h3.span.text)
        except:
            player_status.append('Indian player')
            
    team_list=pd.DataFrame({'name':name,'player_status':player_status,'player_stat':player_stat})
    
    country=[]
    age=[]
    Major_team_played=[]

    for i in team_list['player_stat']:
        URL = i
        r = requests.get(URL)
        soup = BeautifulSoup(r.content, 'html5lib') 
        nation=soup.find_all('b')
        country.append(nation[0].text)

        ages=soup.find_all("p",{"class":"ciPlayerinformationtxt"})
        age.append(int(ages[2].span.text.split()[0]))

        Major_team_played.append(ages[3].span.text[:-1])


    #     print(nation[0].text)

    team_list['country']=country
    team_list['Age']=age
    team_list['Major_team']=Major_team_played
    
    
    team_list['uncapped']=np.where((team_list['Major_team']!='India') & (team_list['player_status']=='Indian player'), 'Yes','No')
    
    Specialist=[]
    Batting_style=[]
    Bowling_style=[]

    for i in team_list['player_stat']:
        URL = i
        r = requests.get(URL)
        soup = BeautifulSoup(r.content, 'html5lib') 
        ages=soup.find_all("p",{"class":"ciPlayerinformationtxt"})

        if ages[4].span.text in ['Wicketkeeper batsman','Wicketkeeper','Top-order batsman','Batsman','Bowler','Allrounder','Batting allrounder','Bowling allrounder','Opening batsman','Middle-order batsman']:
    #         print (ages[0].span.text)
            Specialist.append(ages[4].span.text)
            Batting_style.append(ages[5].span.text)
            try:
                Bowling_style.append(ages[6].span.text)
            except:
                Bowling_style.append("No bowling")

        else :
            if ages[5].span.text in ['Wicketkeeper batsman','Wicketkeeper','Top-order batsman','Batsman','Bowler','Allrounder','Batting allrounder','Bowling allrounder','Opening batsman','Middle-order batsman']:
    #             print (ages[0].span.text)
                Specialist.append(ages[5].span.text)
                Batting_style.append(ages[6].span.text)
                try:
                    Bowling_style.append(ages[7].span.text)
                except:
                    Bowling_style.append("No bowling")
                    
    Mat_FC=[]
    Runs_FC=[]
    HS_FC=[]
    Bat_avg_FC=[]
    Bat_SR_FC=[]
    Wkts_FC=[]
    Bowl_Ave_FC=[]
    Econ_FC=[]
    Bowl_SR_FC=[]

    Mat_in=[]
    Runs_in=[]
    HS_in=[]
    Bat_avg_in=[]
    Bat_SR_in=[]
    Wkts_in=[]
    Bowl_Ave_in=[]
    Econ_in=[]
    Bowl_SR_in=[]
    Played_int=[]

    for i in team_list['player_stat']:

        try:
            print(i)
            perf=pd.read_html(i)

        except:

            Mat_FC.append(0)
            Runs_FC.append(0)
            HS_FC.append(0)
            Bat_avg_FC.append(0)
            Bat_SR_FC.append(0)
            Wkts_FC.append(0)
            Bowl_Ave_FC.append(0)
            Econ_FC.append(0)
            Bowl_SR_FC.append(0)

            Played_int.append('no')
            Mat_in.append(0)
            Runs_in.append(0)
            HS_in.append(0)
            Bat_avg_in.append(0)
            Bat_SR_in.append(0)
            Wkts_in.append(0)
            Bowl_Ave_in.append(0)
            Econ_in.append(0)
            Bowl_SR_in.append(0)

            continue

        Bat=perf[0][perf[0]['Unnamed: 0']=='T20s']
        Bowl=perf[1][perf[1]['Unnamed: 0']=='T20s']

        Bat_in=perf[0][perf[0]['Unnamed: 0']=='T20Is']
        Bowl_in=perf[1][perf[1]['Unnamed: 0']=='T20Is']

        if ((Bat.empty) & (Bat.empty) & (Bat.empty) & (Bat.empty)):
            Mat_FC.append(0)
            Runs_FC.append(0)
            HS_FC.append(0)
            Bat_avg_FC.append(0)
            Bat_SR_FC.append(0)
            Wkts_FC.append(0)
            Bowl_Ave_FC.append(0)
            Econ_FC.append(0)
            Bowl_SR_FC.append(0)

            Played_int.append('no')
            Mat_in.append(0)
            Runs_in.append(0)
            HS_in.append(0)
            Bat_avg_in.append(0)
            Bat_SR_in.append(0)
            Wkts_in.append(0)
            Bowl_Ave_in.append(0)
            Econ_in.append(0)
            Bowl_SR_in.append(0)


        else:
            Played_int.append('no')
            Mat_FC.append(Bat['Mat'].values[0])
            Runs_FC.append(Bat['Runs'].values[0])
            HS_FC.append(Bat['HS'].values[0])
            Bat_avg_FC.append(Bat['Ave'].values[0])
            Bat_SR_FC.append(Bat['SR'].values[0])
            Wkts_FC.append(Bowl['Wkts'].values[0])
            Bowl_Ave_FC.append(Bowl['Ave'].values[0])
            Econ_FC.append(Bowl['Econ'].values[0])
            Bowl_SR_FC.append(Bowl['SR'].values[0])

            try:
                Played_int.append('yes')
                Mat_in.append(Bat_in['Mat'].values[0])
                Runs_in.append(Bat_in['Runs'].values[0])
                HS_in.append(Bat_in['HS'].values[0])
                Bat_avg_in.append(Bat_in['Ave'].values[0])
                Bat_SR_in.append(Bat_in['SR'].values[0])
                Wkts_in.append(Bowl_in['Wkts'].values[0])
                Bowl_Ave_in.append(Bowl_in['Ave'].values[0])
                Econ_in.append(Bowl_in['Econ'].values[0])
                Bowl_SR_in.append(Bowl_in['SR'].values[0])

            except:
                Mat_in.append(0)
                Runs_in.append(0)
                HS_in.append(0)
                Bat_avg_in.append(0)
                Bat_SR_in.append(0)
                Wkts_in.append(0)
                Bowl_Ave_in.append(0)
                Econ_in.append(0)
                Bowl_SR_in.append(0)


    # team_list['Played_international']=Played_int
    team_list['Specialist']=Specialist
    team_list['Batting_style']=Batting_style
    team_list['Bowling_style']=Bowling_style

    # team_list['country']=country
    team_list['Mat_in']=Mat_in
    team_list['Runs_in']=Runs_in
    team_list['HS_in']=HS_in
    team_list['Bat_avg_in']=Bat_avg_in
    team_list['Bat_SR_in']=Bat_SR_in
    team_list['Wkts_in']=Wkts_in
    team_list['Bowl_Ave_in']=Bowl_Ave_in
    team_list['Econ_in']=Econ_in
    team_list['Bowl_SR_in']=Bowl_SR_in
    # team_list['Played_int']=Played_int
    team_list['Mat_FC']=Mat_FC
    team_list['Runs_FC']=Runs_FC
    team_list['HS_FC']=HS_FC
    team_list['Bat_avg_FC']=Bat_avg_FC
    team_list['Bat_SR_FC']=Bat_SR_FC
    team_list['Wkts_FC']=Wkts_FC
    team_list['Bowl_Ave_FC']=Bowl_Ave_FC
    team_list['Econ_FC']=Econ_FC
    team_list['Bowl_SR_FC']=Bowl_SR_FC

    team_list['Skill']=np.where(team_list['Specialist'].str.lower().str.contains('allrounder'),'Allrounder',team_list['Specialist'])
    team_list['Skill']=np.where((team_list['Specialist'].str.lower().str.contains('batsman')) & (~team_list['Specialist'].str.lower().str.contains('wicket')),'batsman',team_list['Skill'])

    team_list=team_list[team_list['player_status']!='withdrawn player']
    
    return team_list


In [19]:
team_list=team_details_scraping(URL)

https://www.espncricinfo.com//ci/content/player/28081.html
https://www.espncricinfo.com//ci/content/player/1083030.html
https://www.espncricinfo.com//ci/content/player/51439.html
https://www.espncricinfo.com//ci/content/player/447261.html
https://www.espncricinfo.com//ci/content/player/32966.html
https://www.espncricinfo.com//ci/content/player/662973.html
https://www.espncricinfo.com//ci/content/player/44828.html
https://www.espncricinfo.com//ci/content/player/288284.html
https://www.espncricinfo.com//ci/content/player/40618.html
https://www.espncricinfo.com//ci/content/player/234675.html
https://www.espncricinfo.com//ci/content/player/290716.html
https://www.espncricinfo.com//ci/content/player/1048813.html
https://www.espncricinfo.com//ci/content/player/694209.html
https://www.espncricinfo.com//ci/content/player/542023.html
https://www.espncricinfo.com//ci/content/player/33141.html
https://www.espncricinfo.com//ci/content/player/502714.html
https://www.espncricinfo.com//ci/content/pla

In [20]:
team_list

,name,player_status,player_stat,country,Age,Major_team,uncapped,Specialist,Batting_style,Bowling_style,...,Mat_FC,Runs_FC,HS_FC,Bat_avg_FC,Bat_SR_FC,Wkts_FC,Bowl_Ave_FC,Econ_FC,Bowl_SR_FC,Skill
0,MS Dhoni,captain/keeper,https://www.espncricinfo.com//ci/content/playe...,India,39,India,No,Wicketkeeper batsman,Right-hand bat,Right-arm medium,...,320,6665,84*,39.91,135.66,0,-,12.50,-,Wicketkeeper batsman
1,KM Asif,Indian player,https://www.espncricinfo.com//ci/content/playe...,India,27,Chennai Super Kings,Yes,Bowler,Right-hand bat,Right-arm medium,...,10,1,1*,-,33.33,11,20.81,7.63,16.3,Bowler
2,Dwayne Bravo,overseas player,https://www.espncricinfo.com//ci/content/playe...,West Indies,36,West Indies,No,Allrounder,Right-hand bat,Right-arm medium,...,465,6324,70*,23.95,127.01,506,24.56,8.23,17.9,Allrounder
3,Deepak Chahar,Indian player,https://www.espncricinfo.com//ci/content/playe...,India,28,India,No,Bowler,Right-hand bat,Right-arm medium,...,78,240,55*,12.00,133.33,98,20.68,7.33,16.9,Bowler
4,Piyush Chawla,Indian player,https://www.espncricinfo.com//ci/content/playe...,India,31,India,No,Allrounder,Left-hand bat,Legbreak,...,240,1361,45*,14.79,125.43,255,24.4,7.47,19.6,Allrounder
5,Sam Curran,overseas player,https://www.espncricinfo.com//ci/content/playe...,England,22,England,No,Allrounder,Left-hand bat,Left-arm medium-fast,...,72,874,55*,19,133.63,68,29.35,8.58,20.5,Allrounder
6,Faf du Plessis,overseas player,https://www.espncricinfo.com//ci/content/playe...,South Africa,36,South Africa,No,Middle-order batsman,Right-hand bat,Legbreak,...,222,5431,119,29.67,127.54,50,18.34,6.96,15.8,batsman
7,Josh Hazlewood,overseas player,https://www.espncricinfo.com//ci/content/playe...,Australia,29,Australia,No,Bowler,Left-hand bat,Right-arm fast-medium,...,37,23,12*,11.50,88.46,43,24.74,7.5,19.7,Bowler
8,Imran Tahir,overseas player,https://www.espncricinfo.com//ci/content/playe...,South Africa,41,South Africa,No,Bowler,Right-hand bat,Legbreak googly,...,301,241,23,10.04,106.16,380,19.69,6.96,16.9,Bowler
9,Ravindra Jadeja,Indian player,https://www.espncricinfo.com//ci/content/playe...,India,31,India,No,Allrounder,Left-hand bat,Slow left-arm orthodox,...,240,2333,48,22,123.04,160,30.5,7.64,23.9,Allrounder


In [ ]:
team_list.to_csv('csk.csv')